In [105]:
import pandas as pd 
from pathlib import Path 

school_file_to_load = Path ("PyCitySchools/Resources/schools_complete.csv")
student_file_to_load = Path ("PyCitySchools/Resources/students_complete.csv")


school_file = pd.read_csv(school_file_to_load)
student_file = pd.read_csv (student_file_to_load)
school_file.head()
# school_file.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [21]:
#Merge Files 
combined_files = pd.merge(school_file, student_file, on=["school_name"])
combined_files.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [22]:
#total students
total_students = combined_files.groupby('school_name')['Student ID'].nunique()
total_students

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: Student ID, dtype: int64

In [101]:
total_schools = combined_files['school_name'].nunique()
total_students = len(combined_files) 
total_budget = combined_files['budget'].sum()
average_math_score = combined_files['math_score'].mean()
average_reading_score = combined_files['reading_score'].mean()

In [102]:
passing_math_percentage = (len(combined_files[combined_files['math_score'] >= 70]) / len(combined_files)) * 100
passing_reading_percentage = (len(combined_files[combined_files['reading_score'] >= 70]) / len(combined_files)) * 100
overall_passing_percentage = (len(combined_files[(combined_files['math_score'] >= 70) & (combined_files['reading_score'] >= 70)]) / len(combined_files)) * 100

In [103]:
district_summary = pd.DataFrame({
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [average_math_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Math': [passing_math_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Overall Passing': [overall_passing_percentage]
})


In [104]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,82932329558,78.985371,81.87784,74.980853,85.805463,65.172326


In [93]:
total_students

39170

In [48]:
school_summary = combined_files.groupby('school_name').agg({
    'type': 'first',
    'Student ID': 'nunique', 
    'budget': 'first',  
    'math_score': 'mean',
    'reading_score': 'mean'
})

school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Student ID']


school_summary['% Passing Math'] = (combined_files[combined_files['math_score'] >= 70].groupby('school_name')['Student ID'].nunique() / school_summary['Student ID']) * 100
school_summary['% Passing Reading'] = (combined_files[combined_files['reading_score'] >= 70].groupby('school_name')['Student ID'].nunique() / school_summary['Student ID']) * 100
school_summary['% Overall Passing'] = (combined_files[(combined_files['math_score'] >= 70) & (combined_files['reading_score'] >= 70)].groupby('school_name')['Student ID'].nunique() / school_summary['Student ID']) * 100
school_summary.reset_index(inplace=True)

school_summary.columns = ['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']

print(school_summary)


              School Name School Type  Total Students  Total School Budget  \
0      Bailey High School    District            4976              3124928   
1     Cabrera High School     Charter            1858              1081356   
2    Figueroa High School    District            2949              1884411   
3        Ford High School    District            2739              1763916   
4     Griffin High School     Charter            1468               917500   
5   Hernandez High School    District            4635              3022020   
6      Holden High School     Charter             427               248087   
7       Huang High School    District            2917              1910635   
8     Johnson High School    District            4761              3094650   
9        Pena High School     Charter             962               585858   
10  Rodriguez High School    District            3999              2547363   
11    Shelton High School     Charter            1761           

In [50]:
top_schools = school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)
top_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,1043130,83.418349,83.848930,638.0,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,1319574,83.274201,83.989488,578.0,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,585858,83.839917,84.044699,609.0,94.594595,95.945946,90.540541


In [52]:
bottom_schools = school_summary.sort_values(by='% Overall Passing').head(5)
bottom_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,2547363,76.842711,80.744686,637.0,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,53.204476
7,Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,53.539172


In [55]:
math_score_by_grade = combined_files.groupby(['school_name', 'grade'])['math_score'].mean().reset_index()
math_score_by_gradez

,school_name,grade,math_score
0,Bailey High School,10th,76.996772
1,Bailey High School,11th,77.515588
2,Bailey High School,12th,76.492218
3,Bailey High School,9th,77.083676
4,Cabrera High School,10th,83.154506
5,Cabrera High School,11th,82.765560
6,Cabrera High School,12th,83.277487
7,Cabrera High School,9th,83.094697
8,Figueroa High School,10th,76.539974
9,Figueroa High School,11th,76.884344


In [56]:
reading_score_by_grade = combined_files.groupby(['school_name', 'grade'])['reading_score'].mean().reset_index()
reading_score_by_grade

,school_name,grade,reading_score
0,Bailey High School,10th,80.907183
1,Bailey High School,11th,80.945643
2,Bailey High School,12th,80.912451
3,Bailey High School,9th,81.303155
4,Cabrera High School,10th,84.253219
5,Cabrera High School,11th,83.788382
6,Cabrera High School,12th,84.287958
7,Cabrera High School,9th,83.676136
8,Figueroa High School,10th,81.408912
9,Figueroa High School,11th,80.640339


In [72]:
per_student_budget = combined_files['budget']/total_students
per_student_budget

0        48.778019
1        48.778019
2        48.778019
3        48.778019
4        48.778019
           ...    
39165    26.630840
39166    26.630840
39167    26.630840
39168    26.630840
39169    26.630840
Name: budget, Length: 39170, dtype: float64

In [ ]:
school_spending_df = 

In [99]:
#Scores by School Spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], bins=spending_bins, labels=labels)

spending_math_scores = school_summary.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,82.529188,620.066667,80.987054,89.219023,73.27756
$585-630,NaN,NaN,NaN,NaN,NaN
$630-645,NaN,NaN,NaN,NaN,NaN
$645-680,NaN,NaN,NaN,NaN,NaN


In [85]:
#Scores by School Size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary ['School Size'] = pd.cut(school_summary['Total Students'], bins=size_bins, labels=labels)

size_math_scores = school_summary.groupby('School Size')['Average Math Score'].mean()
size_reading_scores = school_summary.groupby('School Size')['Average Reading Score'].mean()
size_passing_math = school_summary.groupby('School Size')['% Passing Math'].mean()
size_passing_reading = school_summary.groupby('School Size')['% Passing Reading'].mean()
overall_passing_size = school_summary.groupby('School Size')['% Overall Passing'].mean()


size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": overall_passing_size
})

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.929843,595.000,93.550225,96.099437,89.883853
Medium (1000-2000),83.864438,605.600,93.599695,96.790680,90.621535
Large (2000-5000),81.344493,635.375,69.963361,82.766634,58.286003


In [88]:
#Scores by School Type
type_math_scores = school_summary.groupby('School Type')['Average Math Score'].mean()
type_reading_scores = school_summary.groupby('School Type')['Average Reading Score'].mean()
type_passing_math = school_summary.groupby('School Type')['% Passing Math'].mean()
type_passing_reading = school_summary.groupby('School Type')['% Passing Reading'].mean()
overall_passing_type = school_summary.groupby('School Type')['% Overall Passing'].mean()

type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": overall_passing_type
})

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.896421,599.500000,93.620830,96.586489,90.432244
District,80.966636,643.571429,66.548453,80.799062,53.672208
